In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [1]:
from lightfm.data import *
from lightfm import LightFM
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse
import gc
import lightfm

In [3]:
from load_functions import *
from transform_functions import *
from score_functions import *
from i2i_functions import *
from attributes_recs import *
from learn_to_rank import *
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
test_users = pd.read_csv('test_users.csv', dtype=str, names=['user_id'],header=0)
users_df = load_users('users.csv', test_users)
orgs_df = load_orgs('organisations.csv')
combined_features = load_combined(["rubrics.csv","features.csv"], orgs_df=orgs_df)
test_users_with_city = users_df[users_df.in_test==1][['user_id','city']]
print(len(test_users),len(users_df))

16967 1252801


In [5]:
reviews = load_reviews("reviews.csv", users_df=users_df, orgs_df=orgs_df)
orgs_df=enrich_orgs(orgs_df, reviews)
print(f"{len(reviews)}")
reviews.head()

3640835


,user_id,org_id,rating,ts,aspects,good,user_city,in_test,org_city,travel
0,16998268288908323644,7184895086928047809,2.0,105,[],0,msk,0,msk,0
1,3121447338909258868,7184895086928047809,5.0,464,[],1,msk,0,msk,0
2,1970649778250883025,7184895086928047809,3.0,789,[],0,msk,0,msk,0
3,7554889464530643866,7184895086928047809,4.0,936,[],1,msk,0,msk,0
4,15907910894057053620,7184895086928047809,1.0,1143,[],0,msk,0,msk,0


In [6]:
test_users_df = test_users_with_city\
    .merge(reviews[reviews.good>0][['user_id','org_id']], on='user_id', how='left', indicator=True)
good_test_users = test_users_df[test_users_df["_merge"]=='both']\
    .groupby(['user_id','city'])\
    .agg(org_id=pd.NamedAgg("org_id", aggfunc=list)).reset_index()

empty_test_users = test_users_df[test_users_df["_merge"]=='left_only'][['user_id','city']]
empty_test_users['org_id'] = [[] for _ in empty_test_users.itertuples()]
final_test_users_df = pd.concat([good_test_users, empty_test_users], ignore_index=True)
del test_users_df
del good_test_users
del empty_test_users
print(len(final_test_users_df), len(test_users_with_city))

16967 16967


In [24]:
aspect_transformer = TfidfTransformer(reviews.aspects)
feature_transformer = TfidfTransformer(orgs_df.combined_id)

In [8]:
dataset.fit(users=users_df['user_id'], items=orgs_df['org_id'])

In [20]:
org_reviews = reviews.merge(orgs_df.drop(columns='city'), on='org_id',suffixes=("","_org"))

In [56]:
user_features = pd.read_csv('user_featires.csv',dtype={'user_id'})

In [57]:
user_features.head()

,Unnamed: 0,user_id,user_n_reviews,user_n_plus_reviews,user_n_minus_reviews,user_n_travel_reviews,user_mean_review_rating,user_mean_org_rating,user_mean_plus_rating,user_mean_minus_rating,...,373,374,375,376,377,378,38,380,53,97
0,0,1.000001e+19,3.0,3.0,0.0,2.0,4.333333,1.505828,1.505828,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.000002e+19,3.0,3.0,0.0,0.0,5.000000,3.938900,3.938900,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1.000002e+19,1.0,1.0,0.0,0.0,5.000000,4.552575,4.552575,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1.000003e+19,10.0,9.0,1.0,0.0,4.600000,4.323264,4.349137,4.090400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1.000004e+19,7.0,5.0,2.0,0.0,3.571429,4.180302,4.251718,4.001763,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
def pairs(reviews):
    for row in reviews[reviews['rating']>4].itertuples():
        yield (row.user_id, row.org_id)

In [15]:
interactions = dataset.build_interactions(pairs(reviews))

In [ ]:
dataset.